In [52]:
# Generales.
import pandas as pd
import numpy as np

# Quantum.
from neal import SimulatedAnnealingSampler
from dimod import BQM

# Actividad: Modelización y resolución con Dwave

### Descripción

Se partirá del siguiente mapa:

<img src="img/grafo.png" alt="drawing" width="400"/>

Cada uno de estos nodos representa una ciudad que, como podemos ver, están unidas a través de una serie de carreteras (nótese que no tienen por qué existir carreteras entre todas las ciudades).

El objetivo será encontrar la **ruta de mínima distancia** que cumpla las siguientes características:
- El inicio de la ruta y el final debe ser el nodo 0.
- La segunda ciudad que vamos a visitar (sin contar el origen) debe ser la ciudad 2.
- Todas las ciudades deben ser visitadas.
- Solo podemos ir una vez a cada una de las ciudades.

Como podemos ver, no se ha indicado la distancia del nodo 0 al nodo 2. Da a dicha distancia la longitud que te parezca más acertada.

### Resolución

Se define la matriz $D$ con las distancias asociadas a las transiciones entre los nodos. Como el grafo no es dirigido, la matriz es simétrica respecto a la diagonal.

Además se penalizan todas las transiciones inválidas (no existentes en el grafo) para evitar que sean tenidas en cuenta por el algoritmo de optimización. En esta penalización se incluye a la transición del nodo 0 al 2, que si bien existe en el grafo, se busca evitarla en la solución.

In [53]:
# Inicializar los 5 nodos e instantes.
nodos = range(0,5)
instantes = range(0,5)

# Penalización por transición no válida.
p = 100

# Inicializar matriz D.
D = [
    [p,3,p,4,2],
    [3,p,3,4,p],
    [p,3,p,1,6],
    [4,4,1,p,p],
    [2,p,6,p,p]]

# Mostrar matriz D
pd.DataFrame(D,nodos,nodos)

,0,1,2,3,4
0,100,3,100,4,2
1,3,100,3,4,100
2,100,3,100,1,6
3,4,4,1,100,100
4,2,100,6,100,100


Para definir el problema en términos de QUBO se utilizan **en principio** 25 variables binarias $x_{it}$, donde $i$ representa el nodo (del 0 al 4) y $t$ es el instante del camino en el cual ese nodo fue visitado (un instante del 0 al 4 para cada nodo).

La función a minimizar quedaría de la siguiente forma:

$f(\vec{x}) = \sum_{i=0}^{4} \sum_{j=0}^{4} d_{ij} \sum_{t=0}^{4}x_{i,t}x_{j, t+1 \mod 5} $

Es decir, hay 1 término cuadrático por cada producto entre la variable asociada a cada posible nodo $i$ elegido en cada posible instante $t$ ($x_{i,t}$), multiplicada por la variable asociada a cada posible nodo $j$ elegido en el instante siguiente $t+1$ ($x_{j,t+1 \mod 5}$) (el instante se expresa en módulo 5 para forzar a que el instante 4 se deba multiplicar con el 0, porque tras haber recorrido todos los nodos se vuelve al nodo inicial). $d_{ij}$ hace referencia a la distancia del camino entre el nodo $i$ y el $j$ tomada de la matriz $D$ (este valor actúa como el peso de cada término). 

Para cumplir los criterios de que:
- Se debe visitar un nodo en cada instante.
- Se deben visitar todos los nodos 1 vez.

hay que agregar restricciones, las cuales pueden ser:
- $(\sum_{i=0}^{4} x_it) - 1 = 0,  \forall{t}$  (Por cada instante debe haber exactamente 1 nodo asociado. Esto genera 5 restricciones, una para cada instante $t$).
- $(\sum_{t=0}^{4} x_it) - 1 = 0,  \forall{i}$  (Por cada nodo debe haber exactamente 1 instante asociado. Esto genera 5 restricciones, una para cada nodo $i$).

La siguiente tabla muestra las variables que se utilizarían en principío:
| t\i | 0 | 1 | 2 | 3 | 4 |
| --- | --- | --- | --- | --- | --- |
| **0** | $x_{00}$ | $x_{10}$ | $x_{20}$ | $x_{30}$ | $x_{40}$ |
| **1** | $x_{01}$ | $x_{11}$ | $x_{21}$ | $x_{31}$ | $x_{41}$ |
| **2** | $x_{02}$ | $x_{12}$ | $x_{22}$ | $x_{32}$ | $x_{42}$ |
| **3** | $x_{03}$ | $x_{13}$ | $x_{23}$ | $x_{33}$ | $x_{43}$ |
| **4** | $x_{04}$ | $x_{14}$ | $x_{24}$ | $x_{34}$ | $x_{44}$ |

El problema se puede modelar como un lagrangiano a minimizar:

$L(\vec{x}, \alpha) = f(\vec{x}) - \alpha g(\vec{x})$

Donde $f(\vec{x})$ es la función definida previamente (con los términos cuadráticos), $\alpha$ es el multiplicador de Lagrange y $g(\vec{x})$ es la suma del lado izquierdo de la igualdad de las 10 restricciones.

Se traduce el problema al modelo QUBO usando la clase BQM:

In [60]:
# Devuelve el nombre de la variable en base al nodo e instante. Ejemplo x_31.
def obtener_variable(i, t):
    return "x_" + str(i) + str(t)

# Modelo QUBO del problema.
L = BQM(vartype='BINARY')

# Se agregan los coeficientes cuadráticos expresados en f.
# Se itera por cada combinación de nodos i, j e instantes t.
for i in nodos:
    for j in nodos:
        for t in instantes:
            x_it = obtener_variable(i,t) # Salida desde nodo i en instante t.
            x_jt1 = obtener_variable(i,(t+1) % 5) # Llegada a nodo j en instante t+1 mod 5.
            d = D[i][j] # Duración del camino entre i y j.
            L.add_quadratic(x_it, x_jt1, d)


# Se define el multiplicador de Lagrange elegido por prueba y error.
alfa = -100

# Se agregan las restricciones de igualdad lineales.

# Restricciones que fuerzan a que cada nodo sea visitado en un único instante:
for i in nodos:
    terminos_restriccion = []
    for t in instantes:
        var = obtener_variable(i,t)
        terminos_restriccion.append((var, 1))
    print(terminos_restriccion)
    L.add_linear_equality_constraint(terminos_restriccion, alfa, -1)

# Restricciones que fuerzan a que en cada instante solo tenga 1 nodo visitado.
for t in instantes:
    terminos_restriccion = []
    for i in nodos:
        var = obtener_variable(i,t)
        terminos_restriccion.append((var, 1))
    print(terminos_restriccion)
    L.add_linear_equality_constraint(terminos_restriccion, alfa, -1)

L

[('x_00', 1), ('x_01', 1), ('x_02', 1), ('x_03', 1), ('x_04', 1)]
[('x_10', 1), ('x_11', 1), ('x_12', 1), ('x_13', 1), ('x_14', 1)]
[('x_20', 1), ('x_21', 1), ('x_22', 1), ('x_23', 1), ('x_24', 1)]
[('x_30', 1), ('x_31', 1), ('x_32', 1), ('x_33', 1), ('x_34', 1)]
[('x_40', 1), ('x_41', 1), ('x_42', 1), ('x_43', 1), ('x_44', 1)]
[('x_00', 1), ('x_10', 1), ('x_20', 1), ('x_30', 1), ('x_40', 1)]
[('x_01', 1), ('x_11', 1), ('x_21', 1), ('x_31', 1), ('x_41', 1)]
[('x_02', 1), ('x_12', 1), ('x_22', 1), ('x_32', 1), ('x_42', 1)]
[('x_03', 1), ('x_13', 1), ('x_23', 1), ('x_33', 1), ('x_43', 1)]
[('x_04', 1), ('x_14', 1), ('x_24', 1), ('x_34', 1), ('x_44', 1)]


BinaryQuadraticModel({'x_00': 200.0, 'x_01': 200.0, 'x_02': 200.0, 'x_03': 200.0, 'x_04': 200.0, 'x_10': 200.0, 'x_11': 200.0, 'x_12': 200.0, 'x_13': 200.0, 'x_14': 200.0, 'x_20': 200.0, 'x_21': 200.0, 'x_22': 200.0, 'x_23': 200.0, 'x_24': 200.0, 'x_30': 200.0, 'x_31': 200.0, 'x_32': 200.0, 'x_33': 200.0, 'x_34': 200.0, 'x_40': 200.0, 'x_41': 200.0, 'x_42': 200.0, 'x_43': 200.0, 'x_44': 200.0}, {('x_01', 'x_00'): 9.0, ('x_02', 'x_00'): -200.0, ('x_02', 'x_01'): 9.0, ('x_03', 'x_00'): -200.0, ('x_03', 'x_01'): -200.0, ('x_03', 'x_02'): 9.0, ('x_04', 'x_00'): 9.0, ('x_04', 'x_01'): -200.0, ('x_04', 'x_02'): -200.0, ('x_04', 'x_03'): 9.0, ('x_10', 'x_00'): -200.0, ('x_11', 'x_01'): -200.0, ('x_11', 'x_10'): 10.0, ('x_12', 'x_02'): -200.0, ('x_12', 'x_10'): -200.0, ('x_12', 'x_11'): 10.0, ('x_13', 'x_03'): -200.0, ('x_13', 'x_10'): -200.0, ('x_13', 'x_11'): -200.0, ('x_13', 'x_12'): 10.0, ('x_14', 'x_04'): -200.0, ('x_14', 'x_10'): 10.0, ('x_14', 'x_11'): -200.0, ('x_14', 'x_12'): -200.0, 

Si bien $L$ en principio tendría 25 variables, se puede agregar implícitamente la restricción de que el nodo $0$ debe ser visitado necesariamente en $t=0$ y el nodo $2$ debe ser visitado en $t=2$ (y ningún otro nodo puede visitarse en esos instantes).

Esto reduciría la tabla de variables de la siguiente forma:
| t\i | 0 | 1 | 2 | 3 | 4 |
| --- | --- | --- | --- | --- | --- |
| **0** | $1$ | $0$ | $0$ | $0$ | $0$ |
| **1** | $0$ | $x_{11}$ | $0$ | $x_{31}$ | $x_{41}$ |
| **2** | $0$ | $0$ | $1$ | $0$ | $0$ |
| **3** | $0$ | $x_{13}$ | $0$ | $x_{33}$ | $x_{43}$ |
| **4** | $0$ | $x_{14}$ | $0$ | $x_{34}$ | $x_{44}$ |

Lo cual reduce las variables a optimizar a 9. Esto se puede conseguir fijando el valor de las variables ya conocidas a 0 o 1 en el modelo BQM.

In [61]:
# Fijar variables a 1 si i=t=0 o i=t=2.
# Fijar variables a 0 si i=0 y t!=0, o i=2 y t!=2
for i in nodos:
    for t in instantes:
        var = obtener_variable(i,t)
        if ((i==0 and t==0) or (i==2 and t==2)):
            L.fix_variable(var, 1)
        elif ((i==0 and t!=0) or (i==2 and t!=2) or (i!=0 and t==0) or (i!=2 and t==2)):
            L.fix_variable(var, 0)

L

BinaryQuadraticModel({'x_11': 200.0, 'x_13': 200.0, 'x_14': 200.0, 'x_31': 200.0, 'x_33': 200.0, 'x_34': 200.0, 'x_41': 200.0, 'x_43': 200.0, 'x_44': 200.0}, {('x_13', 'x_11'): -200.0, ('x_14', 'x_11'): -200.0, ('x_14', 'x_13'): 10.0, ('x_31', 'x_11'): -200.0, ('x_33', 'x_13'): -200.0, ('x_33', 'x_31'): -200.0, ('x_34', 'x_14'): -200.0, ('x_34', 'x_31'): -200.0, ('x_34', 'x_33'): 9.0, ('x_41', 'x_11'): -200.0, ('x_41', 'x_31'): -200.0, ('x_43', 'x_13'): -200.0, ('x_43', 'x_33'): -200.0, ('x_43', 'x_41'): -200.0, ('x_44', 'x_14'): -200.0, ('x_44', 'x_34'): -200.0, ('x_44', 'x_41'): -200.0, ('x_44', 'x_43'): 108.0}, -600.0, 'BINARY')

In [62]:
# Se transforma el BQM al formato QUBO para ser ejecutado por el sampler.
L_qubo = L.to_qubo()[0]

# Optimizar el problema QUBO en el annealing sampler.
sampleset = SimulatedAnnealingSampler().sample_qubo(L_qubo, num_reads = 100, chain_strength = 10) 
sampleset.aggregate()

SampleSet(rec.array([([1, 1, 1, 1, 1, 1, 1, 1, 1], -1073., 100)],
          dtype=[('sample', 'i1', (9,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables(['x_11', 'x_13', 'x_14', 'x_31', 'x_33', 'x_34', 'x_41', 'x_43', 'x_44']), {'beta_range': [np.float64(0.0011552453009332421), np.float64(1.177403859232897)], 'beta_schedule_type': 'geometric', 'timing': {'preprocessing_ns': 857100, 'sampling_ns': 8053400, 'postprocessing_ns': 266100}}, 'BINARY')